# Assignment 2

Deadline: 26.03.2025, 12:00 CET

*Buchkov Viacheslav, ID = 24-742-488, viacheslav.buchkov@uzh.ch*\
*Zhiyi Tang, 21-746-763, zhiyi.tang@uzh.ch*\
*Yaqi Wu, 24-746-000, yaqi.wu@uzh.ch*\
*Yangshun Zhou, 24-746-281, yangshun.zhou@uzh.ch*

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [13]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(
    os.path.dirname(os.getcwd())
)  # Change this path if needed
src_path = os.path.join(project_root, "qpmwp-course\\src")
sys.path.append(project_root)
sys.path.append(src_path)
from src.estimation.covariance import Covariance
from src.estimation.expected_return import ExpectedReturn
from src.optimization.constraints import Constraints
from src.optimization.quadratic_program import QuadraticProgram
from src.helper_functions import load_data_msci

In [14]:
def linearize_turnover_constraint(
    self, x_init: np.ndarray, to_budget: float = float("inf")
) -> None:
    """
    Linearize the turnover constraint in the quadratic programming problem with 3-fold augmentation.

    This method modifies the quadratic programming problem to include a linearized turnover constraint.

    Parameters:
    -----------
    x_init : np.ndarray
        The initial portfolio weights.
    to_budget : float, optional
        The maximum allowable turnover. Defaults to float('inf').

    Notes:
    ------
    - The method updates the problem's objective function coefficients, inequality constraints,
    equality constraints, and bounds to account for the turnover constraint.
    - The original problem data is overridden with the updated matrices and vectors.

    Examples:
    ---------
    >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
    >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
    """

    # Control for tau being unnecessarily large
    # maximum turnover = rebalance full portfolio = sell all and buy maximum possible => ub - lb
    lb_orig = self.problem_data.get("lb")
    ub_orig = self.problem_data.get("ub")
    to_budget = min(to_budget, np.max(ub_orig) - np.min(lb_orig))

    # Dimensions
    n = len(self.problem_data.get("q"))

    # 1. Update the coefficients of the objective function
    P_orig = self.problem_data.get("P")
    q_orig = self.problem_data.get("q")

    # Augment by positive and negative parts => 2 * n auxiliary variables
    # that do not contribute to the mean-variance optim.
    P = np.pad(P_orig, ((0, 2 * n), (0, 2 * n)), constant_values=0)
    q = np.pad(q_orig, (0, 2 * n), constant_values=0)

    # 2. Update the equality constraints
    A_orig = self.problem_data.get("A")
    b_orig = self.problem_data.get("b")

    # Need w = w^0 + w^+ - w^- => linear constraint is w - w^+ + w^- = w^0
    # Ax = b => in b we add w^0
    b = np.concatenate((np.array([b_orig]), x_init))

    # In A in each auxiliary row we need to pick row i -> w_i - w_i^+ + w_i^-
    # => use identity in auxiliary rows
    A = np.pad(A_orig, ((0, n), (0, 2 * n)), constant_values=0)
    A[1:, :n] = np.eye(n)
    A[1:, n : 2 * n] = np.eye(n) * -1
    A[1:, 2 * n :] = np.eye(n)

    # 3. Update the inequality constraints
    G_orig = self.problem_data.get("G")
    h_orig = self.problem_data.get("h")

    # Have of the form Gw <= h
    # need to include the new constr of \sum(w^+ + w^-) <= turnover = `to_budget`
    # thus, need to add in G a new row with 0s for true weights and 1s for all w^+ and w^-
    # this last row of G will correspond to last element of h, which should be `to_budget`
    if G_orig is None:
        # If do not have inequality constr => create from scratch
        G = np.pad(np.ones(2 * n), (n, 0), constant_values=0)[np.newaxis, :]
        h = np.array([to_budget])
    else:
        # Pad otherwise
        G = np.pad(G_orig, ((0, 1), (0, 2 * n)), constant_values=0)
        G[n:, n:] = np.ones(2 * n)

        h = np.concatenate((h_orig, np.array([to_budget])))

    # 4. Update lower and upper bounds
    # As we take w = w^0 + w^+ - w^- => w^- is positive => all are positive
    lb = np.pad(lb_orig, (0, 2 * n), constant_values=0)
    lb[n : 2 * n] = 0
    lb[2 * n :] = 0

    # Already have w^0 in our portfolio
    # Maximum for w^+ => maximum can buy (upper_bound - what already have)
    # Maximum for w^- => maximum can buy (what already have + how much we can short)
    ub = np.pad(ub_orig, (0, 2 * n), constant_values=0)
    ub[n : 2 * n] = ub_orig - x_init
    ub[2 * n :] = x_init - lb_orig

    # Override the original matrices
    self.update_problem_data(
        {"P": P, "q": q, "G": G, "h": h, "A": A, "b": b, "lb": lb, "ub": ub}
    )

    return None

## Demo

#### Create P and q

In [15]:
# Load the msci country index data
N = 10
data = load_data_msci(path="../data/", n=N)
X = data["return_series"]

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method="geometric").estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method="pearson").estimate(X=X, inplace=False)

# q, P

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [16]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids=X.columns.tolist())
constraints.add_budget(rhs=1, sense="=")
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P=P.to_numpy(),
    q=q.to_numpy() * 0,
    G=GhAb["G"],
    h=GhAb["h"],
    A=GhAb["A"],
    b=GhAb["b"],
    lb=constraints.box["lower"].to_numpy(),
    ub=constraints.box["upper"].to_numpy(),
    solver="cvxopt",
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)

### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [17]:
MAX_TURNOVER = 0.5

# Prepare initial weights
x_init = pd.Series([1 / X.shape[1]] * X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=MAX_TURNOVER)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get("solution")
ids = constraints.ids
weights = pd.Series(solution.x[: len(ids)], index=ids, name="optimal_weights")
volatility = np.sqrt(weights.to_numpy().T @ P.to_numpy() @ weights.to_numpy())

turnover = np.abs(weights - x_init).sum()

assert turnover <= MAX_TURNOVER, f"Turnover constraint not respected: {turnover} > {MAX_TURNOVER}"

print("Turnover:")
print(turnover)

print("Volatility:")
print(volatility)

Turnover:
0.49961453130807876
Volatility:
0.008833887919600991


In [18]:
pd.concat([x_init.rename("initial_weights"), weights], axis=1)

,initial_weights,optimal_weights
AT,0.1,0.082904
AU,0.1,0.341482
BE,0.1,0.100003
CA,0.1,0.108230
CH,0.1,0.100051
DE,0.1,0.086531
DK,0.1,0.100042
ES,0.1,0.061602
FI,0.1,0.001878
FR,0.1,0.017277


In [19]:
# Use the constant from cbachela/qpmwp-course run to check the turnover
ASSIGNMENT_CONSTANT = 0.49954552248142037

print(f"Deviation vs Assignment Constant is satisfied: {np.isclose(turnover, ASSIGNMENT_CONSTANT, atol=1e-3)}")

Deviation vs Assignment Constant is satisfied: True
